In [1]:
#pip install tensorflow

In [2]:
#pip install keras

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
import tensorflow as tf

Using TensorFlow backend.


In [4]:
abnb = pd.read_csv("airbnb_final.csv")
abnb.head()

,Price,Host_Response_Rate,Accommodates,Bathrooms,Bedrooms,Beds,Guests_Included,Review_Scores_Rating,Entire Home / Apt,Private Room,...,Property_Type)Bed & Breakfast,Property_Type)Condominium,Property_Type)House,Property_Type)Loft,Property_Type)Other,Cancellation_Policy)flexible,Cancellation_Policy)moderate,Cancellation_Policy)strict,Cleaning Fee: No,Cleaning Fee: Yes
0,90,100,2,1.0,0,1,1,94,1,0,...,0,0,0,0,0,0,1,0,0,1
1,140,100,6,1.0,2,2,4,93,1,0,...,0,0,0,0,0,0,1,0,0,1
2,80,99,2,1.0,0,1,1,94,1,0,...,0,0,0,0,0,0,0,1,0,1
3,690,100,7,2.0,5,5,5,90,1,0,...,0,0,0,0,0,0,0,1,0,1
4,90,80,2,1.0,0,1,1,94,1,0,...,0,0,0,0,0,0,1,0,1,0


In [5]:
X = abnb.drop('Price', axis = 'columns')
X.head()

,Host_Response_Rate,Accommodates,Bathrooms,Bedrooms,Beds,Guests_Included,Review_Scores_Rating,Entire Home / Apt,Private Room,Shared Room,...,Property_Type)Bed & Breakfast,Property_Type)Condominium,Property_Type)House,Property_Type)Loft,Property_Type)Other,Cancellation_Policy)flexible,Cancellation_Policy)moderate,Cancellation_Policy)strict,Cleaning Fee: No,Cleaning Fee: Yes
0,100,2,1.0,0,1,1,94,1,0,0,...,0,0,0,0,0,0,1,0,0,1
1,100,6,1.0,2,2,4,93,1,0,0,...,0,0,0,0,0,0,1,0,0,1
2,99,2,1.0,0,1,1,94,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,100,7,2.0,5,5,5,90,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,80,2,1.0,0,1,1,94,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [6]:
sd = StandardScaler().fit(X)
X = sd.transform(X)
X.shape

(30845, 21)

In [7]:
y = abnb['Price']
print(y.shape)
print("\n")
print("Entire - Private - Shared")
print(y)

(30845,)


Entire - Private - Shared
0         90
1        140
2         80
3        690
4         90
        ... 
30840    150
30841     78
30842    245
30843     70
30844    400
Name: Price, Length: 30845, dtype: int64


### Creacion Red Neural

In [8]:
def Red_Prueba(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(18, input_dim = X.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(15, kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='tanh'))
    model.add(Dense(1, kernel_initializer=init, activation='linear'))

    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

model = KerasClassifier(build_fn=Red_Prueba, verbose=0)

### Tunning, utilizando Gridsearch CV

In [9]:
optimizers = ['SGD', 'adam', 'Nadam']
init = ['uniform', 'normal', 'zero']
epochs = [25, 50, 75]
batches = [5, 10, 15]

X_Trial = X[0:500,:] 
y_Trial = y[0:500,]
print(X_Trial.shape, y_Trial.shape)

(500, 21) (500,)


### Intento GridSearchCV, murio el kernel de SageMaker

In [10]:
#Tunnin_Grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
#grid_cv = GridSearchCV(estimator=model, param_grid=Tunnin_Grid, scoring='neg_mean_absolute_error')
#best_grid = grid_cv.fit(X_Trial, y_Trial)

#print("BestGrid: %f using %s" % (best_grid.best_score_, best_grid.best_params_))
#means = best_grid.cv_results_['mean_test_score']
#stds = best_grid.cv_results_['std_test_score']
#params = best_grid.cv_results_A['params']
#for mean, stdev, param in zip(means, stds, params):
    #print("%f (%f) with: %r" % (mean, stdev, param))

### Separacion X y | Train y Test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)
X_test, X_cv, y_test, y_cv = train_test_split(X_test, y_test, test_size=0.3, random_state=100)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_cv.shape, y_cv.shape)

(18507, 21) (18507,) (8636, 21) (8636,) (3702, 21) (3702,)


### Modelos

In [18]:
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
def modelo():
    model = Sequential()
    model.add(Dense(15, input_shape = (21,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
KEstimator = KerasRegressor(build_fn=modelo, epochs=200, batch_size=25, verbose=0)
KF = KFold(n_splits = 10)
results = cross_val_score(KEstimator, X_test, y_test, cv=KF)

In [19]:
print("Resultados de la media y varianza extraídos de los 10 splits de CV")
print("R: %.2f (%.2f) MSE" % (results.mean(), results.std()))
model.summary()

Baseline: -2971.36 (467.27) MSE


In [22]:
def modelo1():
    model = Sequential()
    model.add(Dense(15, input_shape = (21,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
KEstimator = KerasRegressor(build_fn=modelo1, epochs=200, batch_size=25, verbose=0)
KF = KFold(n_splits = 15)
resultados = cross_val_score(KEstimator, X_test, y_test, cv=KF)

print("Resultados de la media y varianza extraídos de los 10 splits de CV")
print("R: %.2f (%.2f) MSE" % (results.mean(), results.std()))
model.summary()

Resultados de la media y varianza extraídos de los 10 splits de CV
R: -2971.36 (467.27) MSE
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                330       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 346
Trainable params: 346
Non-trainable params: 0
_________________________________________________________________


In [23]:
def modelo2():
    model = Sequential()
    model.add(Dense(18, input_shape = (21,), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(12, kernel_initializer='normal', activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
KEstimator = KerasRegressor(build_fn=modelo2, epochs=200, batch_size=25, verbose=0)
KF = KFold(n_splits = 15)
resultados = cross_val_score(KEstimator, X_test, y_test, cv=KF)

print("Resultados de la media y varianza extraídos de los 15 splits de CV")
print("R: %.2f (%.2f) MSE" % (results.mean(), results.std()))
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Resultados de la media y varianza extraídos de los 15 splits de CV
R: -2971.36 (467.27) MSE
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                330       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 346
Trainable params: 346
Non-trainable params: 0
_________________________________________________________________


### Predicción

In [41]:
model = Sequential()
model.add(Dense(25, input_shape = (21,), kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(20, kernel_initializer='normal', activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(18, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(12, kernel_initializer='normal', activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(6, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])
model.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_136 (Dense)            (None, 25)                550       
_________________________________________________________________
dropout_35 (Dropout)         (None, 25)                0         
_________________________________________________________________
dense_137 (Dense)            (None, 20)                520       
_________________________________________________________________
dropout_36 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_138 (Dense)            (None, 18)                378       
_________________________________________________________________
dropout_37 (Dropout)         (None, 18)                0         
_________________________________________________________________
dense_139 (Dense)            (None, 12)                228       
__________

In [47]:
history = model.fit(X_train, y_train, epochs=200, batch_size=25, verbose=0)

In [48]:
_, mae = model.evaluate(X_test, y_test)
print('MAE:', mae)

8636/8636 [==============================] - 0s 31us/step
MAE: 30.964674929768577


In [49]:
_, mae = model.evaluate(X_cv, y_cv)
print('MAE:', mae)

3702/3702 [==============================] - 0s 32us/step
MAE: 30.221572212371743


In [50]:
abnb["Price"].mean()

97.69502350461987